# Use TI SDK For CFS

本例介绍了TI SDK使用CFS做为输入数据源训练Tensorflow模型

### CFS数据上传指引

腾讯云的文件存储（Cloud File Storage, CFS）提供了标准的 NFS 及 CIFS/SMB 文件系统访问协议。使用CFS，您的训练任务时，可以节省数据从远程下载的时间。
您上传数据至CFS的云主机需满足如下条件：
1. 云主机已安装NFS客户端
2. 云主机与CFS处于同一个私有网络环境中

CFS_IP等信息可从腾讯云控制台获得
挂载CFS根目录命令为：sudo mount -t nfs -o vers=4 CFS_IP:/ /localfolder

挂载CFS子目录命令为：
sudo mount -t nfs -o vers=4 CFS_IP:/subfolder /localfolder 

解挂CFS目录命令为：
umount /localfolder

具体数据上传指引如下：
https://cloud.tencent.com/document/product/582/11523

### CFS训练数据做为输入源

您需要向CFS文件系统中上传训练数据，具体步骤如下：
1. 从公共COS数据集中下载训练数据
2. 在CFS文件系统的根目录下创建training-data目录
3. 将下载的heart.csv数据拷贝至training-data目录中

In [1]:
#下载数据集
!wget  https://ti-${REGION}-1300268737.cos.${REGION}.myqcloud.com/training_data/tensorflow/simple/heart.csv

In [2]:
from ti.session import FileSystemInput

#tencentcloud cfs filesystem id, obtained from console.
file_system_id = 'cfs-xxxxxxx'

#specify filesystem directory will mount
file_system_directory_path = '/training-data/'

file_system_type = "cfs"

# file access mode in 'ro'(read-only) or 'rw' (read-write).
file_system_access_mode = 'rw'
train_input = FileSystemInput(file_system_id=file_system_id,
                                directory_path=file_system_directory_path,
                                file_system_type = file_system_type,
                                file_system_access_mode=file_system_access_mode)

### 提交训练任务

In [ ]:
from __future__ import absolute_import, print_function

import sys
from ti import session
from ti.tensorflow import TensorFlow

# 初始化一个session
ti_session = session.Session()

# 授权给TI的服务角色
role = "TIONE_QCSRole"

# 设置算法超级参数
hyperparameters = {'batch_size': 64}


# 创建一个Tensorflow Estimator
tf_estimator = TensorFlow(role=role,
                          train_instance_count=1,
                          train_instance_type='TI.SMALL2.1core2g',
                          py_version='py3',
                          script_mode=True,
                          hyperparameters=hyperparameters,
                          framework_version='2.0.0b',
                          subnet_id = "subnet-yyyyyy",
                          security_group_ids = ["sg-xxxxxx"],
                          entry_point='train.py',
                          source_dir='code')

# 提交Tensorflow训练任务
tf_estimator.fit({'training': train_input})